In [1]:
%load_ext autoreload
%autoreload 2

import geopandas as gpd
import numpy as np
import pandas as pd
import re
import copy
from powergenome.generators import GeneratorClusters
from powergenome.GenX import reduce_time_domain, add_misc_gen_values
from powergenome.params import DATA_PATHS

from powergenome.external_data import (
    make_demand_response_profiles,
    make_generator_variability,
)

from powergenome.load_profiles import (
    make_load_curves, 
    add_load_growth, 
    make_final_load_curves, 
    make_distributed_gen_profiles,
)
from powergenome.external_data import make_demand_response_profiles
from powergenome.util import (
    build_scenario_settings,
    init_pudl_connection,
    load_settings,
    check_settings,
    reverse_dict_of_lists,
    remove_feb_29,
    snake_case_str,
    snake_case_col
)

from pathlib import Path

from powergenome.generators import load_ipm_shapefile
from powergenome.GenX import (
    add_cap_res_network,
    add_co2_costs_to_o_m,
    add_misc_gen_values,
    calculate_partial_CES_values,
    check_resource_tags,
    create_policy_req,
    create_regional_cap_res,
    fix_min_power_values,
    hydro_energy_to_power,
    max_cap_req,
    min_cap_req,
    network_line_loss,
    network_max_reinforcement,
    network_reinforcement_cost,
    reduce_time_domain,
    round_col_values,
    set_int_cols,
    set_must_run_generation,
)
from powergenome.transmission import (
    agg_transmission_constraints,
    transmission_line_distance,
)
from powergenome.fuels import fuel_cost_table

from powergenome.run_powergenome_multiple_outputs_cli import main

from powergenome.external_data import (
    insert_user_tx_costs,
    load_user_tx_costs,
    make_generator_variability,
)

pd.options.display.max_columns = 200

In [2]:
import pandas as pd
import numpy as np
import os
import sqlite3
import shutil
import datetime
import matplotlib.pyplot as plt

# Outstanding Issues:
- Lifetime in all_gens
- Add more battery ratios (just use NREL ones)

# Temporary Things:
### Hardcoding Battery Ratios

Powergenome assumes the capacity expansion model can co-optimize battery power and energy capacities. Currently, Temoa only optimizes for power capacity and, accordingly, defines different battery technologies for different energy-to-capacity ratios.

Here, we pre-define several energy-to-capacity ratios that we will use in the Temoa model.

In [3]:
battery_ratios = [2, 4, 6, 8, 10] # energy:capacity ratios

# Non-Temporary Things

Define the scenario of interest. This should be from the list of scenarios in the CONUS_extra_inputs/conus_scenario_inputs.csv file ("case_id" column).

In [28]:
scenario = 'current_policies_52_week'
scenario = 'base_52_week'
folder = "settings-atb2023"

In [29]:
cwd = Path.cwd()

settings_path = (
    cwd.parent / "Git-Results-Comparison" / "MIP_results_comparison" / "case_settings" / "26-zone" / folder 
)

settings = load_settings(settings_path)
settings["input_folder"] = settings_path.parent / settings["input_folder"]
scenario_definitions = pd.read_csv(
    settings["input_folder"] / settings["scenario_definitions_fn"]
)

pudl_engine, pudl_out, pg_engine = init_pudl_connection(
    freq="AS",
    start_year=min(settings.get("eia_data_years")),
    end_year=max(settings.get("eia_data_years")),
    )

check_settings(settings, pg_engine)
scenario_settings = build_scenario_settings(settings, scenario_definitions)




    *****************************
    The ATB technology "NaturalGas_H-Frame CC 95% CCS" listed in your settings file under 'atb_new_gen'
    is not fully specified in the 'cost_multiplier_technology_map' settings parameter.
    Part of the <tech>_<tech_detail> string might be included, but it is best practice to
    include the full name in this format. Check your settings file.
        

    *****************************
    The ATB technology "NaturalGas_H-Frame CC" listed in your settings file under 'atb_new_gen'
    is not fully specified in the 'cost_multiplier_technology_map' settings parameter.
    Part of the <tech>_<tech_detail> string might be included, but it is best practice to
    include the full name in this format. Check your settings file.
        

    *****************************
    The ATB technology "NaturalGas_F-Frame CT" listed in your settings file under 'atb_new_gen'
    is not fully specified in the 'cost_multiplier_technology_map' settings parameter.
    

In [30]:
str(settings.get("input_folder")) + '/' + settings.get("emission_policies_fn")

'/mnt/c/Users/camer/Documents/CMU/Modelling/PowerGenome/Git-Results-Comparison/MIP_results_comparison/case_settings/26-zone/CONUS_extra_inputs/conus_net_zero_emission_limits.csv'

In [31]:
scenario_definitions[scenario_definitions.case_id == scenario]

,case_id,year,flex_resource,ng_price,limitedtransmission,ira,min_cap,existing_clusters,hydrogen_gen,time_reduction,policies,period_years,econ_retirement,wacc,misc_file,carbon_slack,ccs,ccs_credit
75,base_52_week,2027,none,reference,no,no,none,none,greenfield,52_weeks,yes,five,no,standardize_5,no_ramp,base,yes,none
76,base_52_week,2030,none,reference,no,no,none,none,greenfield,52_weeks,yes,five,no,standardize_5,no_ramp,base,yes,none
77,base_52_week,2035,none,reference,no,no,none,none,greenfield,52_weeks,yes,five,no,standardize_5,no_ramp,base,yes,none
78,base_52_week,2040,none,reference,no,no,none,none,greenfield,52_weeks,yes,five,no,standardize_5,no_ramp,base,yes,none
79,base_52_week,2045,none,reference,no,no,none,none,greenfield,52_weeks,yes,five,no,standardize_5,no_ramp,base,yes,none
80,base_52_week,2050,none,reference,no,no,none,none,greenfield,52_weeks,yes,five,no,standardize_5,no_ramp,base,yes,none


In [32]:
results_directory = str(settings_path) + '/Computed_Data_' + scenario +'/' 
isExist = os.path.exists(results_directory)
if not isExist:
    os.makedirs(results_directory)


OK, the scenario settings are now properly set up. 

Let's start defining some local variables to help us w/ our translation layer.

In [33]:
all_periods = []
for p in scenario_settings.keys():
    if scenario in scenario_settings[p].keys():
        all_periods.append(p)


In [34]:
all_periods = [all_periods[0]]

In [35]:
start_year = all_periods[0]

## Load Generator and Fuels Data for All Model Years

We'll now proceed with saving the data for existing generators and new generators. 

Four dataframes will be created:
1. `existing_gens` contains the data characterizing all existing generators.
2. `new_gens` contains the data characterizing new resource options.
3. `all_gens` contains data characterizing both new and existing generators.
4. `gen_variability` contains time-series data for all generators used to generate hourly capacity factors. 

Although the `all_gens` dataframe may be redundant, it is needed as an input to the function that generates the variability data.

We output these four dataframes for every model year and save them as a dictionary, e.g., `new_gens` is a dictionary with keys corresponding to each model year, and the respective values are the dataframes described above.

In [36]:
def calculate_annuity(wacc, n_periods, principal):
    wacc = float(wacc)
    n_periods = int(n_periods)
    principal = float(principal)
    numerator = wacc * principal
    denominator = 1-pow((1+wacc),(-n_periods))
    return abs(numerator / denominator)

def calculate_capex(wacc, n_periods, annuity):
    numerator = annuity * (1 - pow((1+wacc),-n_periods))
    denominator = wacc
    return abs(numerator / denominator)

def determine_capex(df):
    ### There are 2 parts of determining the capex:
    # 1. multiply the stated capex by the regional cost multiplier
    # 2. to that value, the interconnection cost must be added.
    
    for index, row in df.iterrows(): 
        _capex = row.capex_mw * row.regional_cost_multiplier
        # the interconnection cost has different financing terms: 
        # - 60 years
        # - 4.4% WACC
        #wacc = 0.044
        #n_periods = 60
        # Greg has since updated this to 30y at 5%
        wacc = 0.05
        n_periods = 30
        
        tx_capex_orig = row.interconnect_capex_mw
        tx_annuity = calculate_annuity(float(wacc), int(n_periods), float(tx_capex_orig))
        df.loc[index,'Interconnection Capex (test)'] = calculate_capex(wacc, n_periods, tx_annuity)
        tx_capex_new = calculate_capex(row.wacc_real, row.cap_recovery_years, tx_annuity)
        df.loc[index,'Interconnection Capex'] = tx_capex_new
        _capex += tx_capex_new
        df.loc[index,'Total Capex'] = _capex 
        
    return df


In [37]:
new_gens = dict()
all_gens = dict()
existing_gens = dict()
gen_variability = dict()



for year in all_periods:
    print('\n\n Year: ', year)
    _settings = scenario_settings[year][scenario]
    gc = GeneratorClusters(pudl_engine, pudl_out, pg_engine, _settings)
    all_gens[year] = gc.create_all_generators()
    all_gens[year] = add_misc_gen_values(all_gens[year],_settings)
    # Drop columns that were causing csv parsing issues
    if 'plant_id_eia' in all_gens[year].columns:
        all_gens[year].drop(columns = ['plant_id_eia'], axis=1, inplace=True)
    if 'unit_id_pg' in all_gens[year].columns:
        all_gens[year].drop(columns = ['unit_id_pg'], axis=1, inplace=True)
    # Add interconnection costs
    all_gens[year]['Interconnection Capex (test)'] = 0
    all_gens[year]['Interconnection Capex'] = 0
    all_gens[year]['Total Capex'] = 0
    all_gens[year] = determine_capex(all_gens[year])
    all_gens[year].loc[:,'operating_year'] = int(year)
    # end
    if 'Existing_Cap_MW' in all_gens[year].columns:
        existing_gens[year] = all_gens[year][all_gens[year].Existing_Cap_MW > 0]
        new_gens[year] = all_gens[year][all_gens[year].Existing_Cap_MW.isnull()]
    else:
        existing_gens[year] = all_gens[year].head(0)
        new_gens[year] = copy.copy(all_gens[year])
    new_gens[year].loc[:,'operating_year'] = int(year)
    
    gen_variability[year] = make_generator_variability(all_gens[year])
    gen_variability[year].columns = all_gens[year].Resource
    # This normalizes all values in the small_hydroelectric columns
    # gen_variability[year].update(gen_variability[year].filter(like='small_hydroelectric').apply(lambda x: x / x.max()))
    # This sets the values of all small_hydroelectric columns to 1.0
    gen_variability[year].loc[:, gen_variability[year].columns.str.contains('small_hydroelectric')] = 1.0

    if year == all_periods[0]:
        all_gens_for_fuels = all_gens[year][['Fuel', 'region']]
        all_gens_for_fuels = all_gens_for_fuels[all_gens_for_fuels.Fuel != 'None']
        fuels = fuel_cost_table(gc.fuel_prices, generators=all_gens[year], settings=_settings)
        elec_demand = make_final_load_curves(pg_engine, _settings)
        elec_demand.loc[:,'periods'] = all_periods[0]
        
    load_curves_year = make_final_load_curves(pg_engine, _settings)
    load_curves_year.loc[:,'periods']=year
    elec_demand = pd.concat([elec_demand, load_curves_year])
    
    _agff = all_gens[year][['Fuel', 'region']]
    _agff = _agff[_agff.Fuel != 'None']
    all_gens_for_fuels = pd.concat([all_gens_for_fuels, _agff])    
    all_gens_for_fuels.reset_index(inplace=True, drop=True)

    _fuels = fuel_cost_table(gc.fuel_prices, generators=all_gens_for_fuels, settings=_settings)
    _fuels_price = _fuels.iloc[1:,:]
    _fuels_price['year'] = year
    _fuels_co2 = _fuels.iloc[0,:]
    
    if year == all_periods[0]:
        fuels_price = _fuels_price
        fuels_co2 = _fuels_co2
        
    fuels_price = pd.concat([fuels_price, _fuels_price])
    fuels_price.reset_index(inplace=True, drop=True)

fuels_co2.name = 'CO2_Intensity_tons_per_MMBtu'




 Year:  2027


Some generators do not have lon/lat data. Check the source file to determine if they should be included in results. 
The affected generators account for 4.5 in balancing authorities: 
['CISO']
The model resource tags {'Retro2_Inv_Cost_per_MWyr', 'Retro1_Inv_Cost_per_MWyr', 'Retro3_Inv_Cost_per_MWyr'} are listed in the settings parameter 'model_tags_name' but are not assigned values for any resources
Selected technology utilitypv capacity in region MISS less than minimum (46042.63888889999 < 50000 MW)
Selected technology utilitypv capacity in region MISW less than minimum (198819.80554839998 < 200000 MW)
Selected technology landbasedwind capacity in region NYUP less than minimum (22018.8375186 < 40000 MW)
Selected technology landbasedwind capacity in region PJMD less than minimum (8839.462477350002 < 20000 MW)
Selected technology utilitypv capacity in region PJME less than minimum (85769.0778322 < 125000 MW)
Selected technology landbasedwind capacity in region SRCA less than minimum (11

In [38]:
year = []
daygroup = []
hour = []
timestep = []
dayweight = []
segfrac = []

representative_periods = dict()
reduced_resource_profile = dict()
reduced_load_profile = dict()
load_curves = dict()

## We need to construct the load_curves for all years...
regions = list(all_gens[start_year].region.unique())
for y in all_periods:
    _settings = scenario_settings[y][scenario]
    print('Year: ', y)
    load_curves[y] = make_final_load_curves(pg_engine, _settings)
    
    (
        reduced_resource_profile[y],
        reduced_load_profile[y],
        _time_series_mapping,
        _representative_point,
    ) = reduce_time_domain(gen_variability[y], load_curves[y], _settings)
    
    representative_periods[y] = _representative_point
    
    if _representative_point is None: # 8760 mode
        print('8760 Mode...')
        n_rp = 1
        n_ts_per_rp = 8760
        n_ts = 8760
        year += [y]*n_ts
        _dw = [365]*n_ts
        daygroup += ['8760_mode']*n_ts
        hour += list(np.arange(n_ts_per_rp))
        timestep += list(np.arange(n_ts_per_rp))
        segfrac += list(np.asarray(_dw) / np.asarray(_dw).sum())
        dayweight += _dw
        
        reduced_load_profile[y] = reduced_load_profile[y][regions] # remove all the weird columns.
        reduced_load_profile[y]['year'] = y
        if y == all_periods[0]:
            dfl = copy.copy(reduced_load_profile[y])
        else:
            dfl = pd.concat([dfl, reduced_load_profile[y]])

        
        reduced_resource_profile[y]['year'] = y
        if y == all_periods[0]:
            dfr = copy.copy(reduced_resource_profile[y])
        else:
            dfr = pd.concat([dfr, reduced_resource_profile[y]])
        
        
    else:
        n_rp = int(reduced_load_profile[y].Rep_Periods.values[0])
        n_ts_per_rp = int(reduced_load_profile[y].Timesteps_per_Rep_Period.values[0])
        n_ts = n_rp * n_ts_per_rp
        time_season = _representative_point.slot.values
    
        year += [y]*n_ts
        _dw = []
        for idx, rp in enumerate(time_season):
            daygroup += [rp] *  n_ts_per_rp
            _dw += [reduced_load_profile[y].Sub_Weights.values[idx]] * n_ts_per_rp
            hour += list(np.arange(n_ts_per_rp))
            if idx == 0:
                start = 0
            else:
                start = timestep[-1] + 1
            timestep += list(np.arange(start, n_ts_per_rp + start))
        segfrac += list(np.asarray(_dw) / np.asarray(_dw).sum())
        dayweight += _dw
        reduced_load_profile[y] = reduced_load_profile[y][regions] # remove all the weird columns.
        reduced_load_profile[y]['year'] = y
        if y == all_periods[0]:
            dfl = copy.copy(reduced_load_profile[y])
        else:
            dfl = pd.concat([dfl, reduced_load_profile[y]])
            
        #reduced_resource_profile[y]['year'] = y
#         if y == all_periods[0]:
#             dfr = copy.copy(reduced_resource_profile[y])
#         else:
#             dfr = pd.concat([dfr, reduced_resource_profile[y]],ignore_index=True)



Year:  2027


In [40]:
a

,region,technology,cluster,Cap_Size,Existing_Cap_MWh,minimum_load_mw,Heat_Rate_MMBTU_per_MWh,Fixed_OM_Cost_per_MWyr,Var_OM_Cost_per_MWh,heat_rate_mmbtu_mwh_iqr,heat_rate_mmbtu_mwh_std,fixed_o_m_mw_std,Min_Power,num_units,capacity_factor,unmodified_cap_size,Existing_Cap_MW,unmodified_existing_cap_mw,profile,Start_Fuel_MMBTU_per_MW,Fuel,Start_Cost_per_MW,CapRes_2,RETRO,New_Build,CapRes_6,ESR_16,ESR_15,ESR_12,CapRes_9,ESR_14,Retro2_Inv_Cost_per_MWyr,CapRes_10,Retro1_Efficiency,ESR_9,ESR_13,MinCapTag_4,gen_is_variable,Retro2_Source,Retro1_Inv_Cost_per_MWyr,THERM,Retro3_Efficiency,MinCapTag_1,ESR_3,ESR_8,CO2_Capture_Rate,Num_RETRO_Sources,HYDRO,Retro2_Efficiency,ESR_2,ESR_1,CO2_Capture_Cost_per_Metric_Ton,LDS,CapRes_7,CapRes_4,Retro1_Source,CapRes_5,MinCapTag_2,Num_VRE_Bins,ESR_10,FLEX,ESR_6,CapRes_8,STOR,ESR_7,CapRes_3,MinCapTag_3,MinCapTag_5,CapRes_1,ESR_5,MUST_RUN,VRE,ESR_4,Retro3_Inv_Cost_per_MWyr,Flexible_Demand_Energy_Eff,Retro3_Source,ESR_11,Resource,cost_case,tech_detail,basis_year,Fixed_OM_Cost_per_MWhyr,capex_mw,capex_mwh,wacc_real,Var_OM_Cost_per_MWh_In,cap_recovery_years,Inv_Cost_per_MWyr,Inv_Cost_per_MWhyr,Max_Cap_MW,Max_Cap_MWh,regional_cost_multiplier,area,lcoe,interconnect_annuity,interconnect_capex_mw,m_popden,spur_miles,spur_capex,spur_inv_mwyr,plant_inv_cost_mwyr,backbone_cost_tonne,co2_o_m_mw,co2_pipeline_annuity_mw,co2_pipeline_capex_mw,storage_cost_tonne,capture_rate,extra_ccs_cost_tonne,tonne_co2_captured_mwh,co2_cost_mwh,variable_CF,R_ID,Self_Disch,Eff_Up,Eff_Down,Min_Duration,Max_Duration,Ramp_Up_Percentage,Ramp_Dn_Percentage,Up_Time,Down_Time,Min_Share_percent,Max_Share_percent,Existing_Charge_Cap_MW,Reg_Cost,Rsv_Cost,Reg_Max,Rsv_Max,gen_energy_source,gen_is_cogen,gen_is_baseload,gen_scheduled_outage_rate,gen_forced_outage_rate,gen_type,gen_ccs_capture_efficiency,gen_ccs_energy_load,_gen_scheduled_outage_rate,_gen_forced_outage_rate,Interconnection Capex (test),Interconnection Capex,Total Capex,operating_year
0,BASN,Biomass,1,0.941,0.0,1.138,15.348,157220.000,5.730,1.650,1.385,0.000,0.0,8.0,0.597,1.575,7.528,12.600,NaN,0.0,waste_biomass,172.369,0.0,0,-1,0.0,0,0,0,0.0,0,0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,1,0,0,0.0,0.0,0,0.0,0,0,0,0,0,0.0,0,0,0.93,0,0,0.0,0,1,0,0,0,0,0,0,BASN_biomass_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.000000,1,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.00,Waste_biomass,False,False,0.0,0.0,Biomass,.,.,0.06,0.04,NaN,NaN,NaN,2027
1,BASN,Biomass,2,0.932,0.0,0.833,9.465,157220.000,3.534,1.106,1.291,0.000,0.0,18.0,0.597,1.561,16.776,28.098,NaN,0.0,waste_biomass,172.369,0.0,0,-1,0.0,0,0,0,0.0,0,0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,1,0,0,0.0,0.0,0,0.0,0,0,0,0,0,0.0,0,0,0.93,0,0,0.0,0,1,0,0,0,0,0,0,BASN_biomass_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.000000,2,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.00,Waste_biomass,False,False,0.0,0.0,Biomass,.,.,0.06,0.04,NaN,NaN,NaN,2027
2,BASN,Conventional Hydroelectric,1,56.580,0.0,0.000,8.843,47048.867,0.000,0.000,0.000,0.000,0.0,44.0,0.356,56.580,2489.52,2489.520,"[0.29812578993421274, 0.2982087954110899, 0.29...",0.0,None,0.000,0.0,0,-1,0.0,0,0,0,0.0,0,0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0,1,0,0,0,0,0,0.0,0.0,0,0.0,0,0,0,0,0,0.0,0,0,0.80,0,0,0.0,0,0,0,0,0,0,0,0,BASN_conventional_hydroelectric_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.389270,3,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.00,Water,False,False,0.0,0.0,Hydro,.,.,0.05,0.05,NaN,NaN,NaN,2027
3,BASN,Conventional Steam Coal,1,294.550,0.0,96.250,10.572,71499.957,1.879,0.312,0.486,4123.244,0.0,4.0,0.305,294.550,1178.2,1178.200,NaN,16.5,mountain_reference_coal,123.906,0.0,0,-1,0.0,0,0,0,0.0,0,0,0.0,0,0,0,0,0,0,0,1,0,0,0,0,0.0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0,0,0

In [41]:
a[a.region == 'BASN']

,region,technology,cluster,Cap_Size,Existing_Cap_MWh,minimum_load_mw,Heat_Rate_MMBTU_per_MWh,Fixed_OM_Cost_per_MWyr,Var_OM_Cost_per_MWh,heat_rate_mmbtu_mwh_iqr,heat_rate_mmbtu_mwh_std,fixed_o_m_mw_std,Min_Power,num_units,capacity_factor,unmodified_cap_size,Existing_Cap_MW,unmodified_existing_cap_mw,profile,Start_Fuel_MMBTU_per_MW,Fuel,Start_Cost_per_MW,CapRes_2,RETRO,New_Build,CapRes_6,ESR_16,ESR_15,ESR_12,CapRes_9,ESR_14,Retro2_Inv_Cost_per_MWyr,CapRes_10,Retro1_Efficiency,ESR_9,ESR_13,MinCapTag_4,gen_is_variable,Retro2_Source,Retro1_Inv_Cost_per_MWyr,THERM,Retro3_Efficiency,MinCapTag_1,ESR_3,ESR_8,CO2_Capture_Rate,Num_RETRO_Sources,HYDRO,Retro2_Efficiency,ESR_2,ESR_1,CO2_Capture_Cost_per_Metric_Ton,LDS,CapRes_7,CapRes_4,Retro1_Source,CapRes_5,MinCapTag_2,Num_VRE_Bins,ESR_10,FLEX,ESR_6,CapRes_8,STOR,ESR_7,CapRes_3,MinCapTag_3,MinCapTag_5,CapRes_1,ESR_5,MUST_RUN,VRE,ESR_4,Retro3_Inv_Cost_per_MWyr,Flexible_Demand_Energy_Eff,Retro3_Source,ESR_11,Resource,cost_case,tech_detail,basis_year,Fixed_OM_Cost_per_MWhyr,capex_mw,capex_mwh,wacc_real,Var_OM_Cost_per_MWh_In,cap_recovery_years,Inv_Cost_per_MWyr,Inv_Cost_per_MWhyr,Max_Cap_MW,Max_Cap_MWh,regional_cost_multiplier,area,lcoe,interconnect_annuity,interconnect_capex_mw,m_popden,spur_miles,spur_capex,spur_inv_mwyr,plant_inv_cost_mwyr,backbone_cost_tonne,co2_o_m_mw,co2_pipeline_annuity_mw,co2_pipeline_capex_mw,storage_cost_tonne,capture_rate,extra_ccs_cost_tonne,tonne_co2_captured_mwh,co2_cost_mwh,variable_CF,R_ID,Self_Disch,Eff_Up,Eff_Down,Min_Duration,Max_Duration,Ramp_Up_Percentage,Ramp_Dn_Percentage,Up_Time,Down_Time,Min_Share_percent,Max_Share_percent,Existing_Charge_Cap_MW,Reg_Cost,Rsv_Cost,Reg_Max,Rsv_Max,gen_energy_source,gen_is_cogen,gen_is_baseload,gen_scheduled_outage_rate,gen_forced_outage_rate,gen_type,gen_ccs_capture_efficiency,gen_ccs_energy_load,_gen_scheduled_outage_rate,_gen_forced_outage_rate,Interconnection Capex (test),Interconnection Capex,Total Capex,operating_year
0,BASN,Biomass,1,0.941,0.0,1.138,15.348,157220.000,5.730,1.650,1.385,0.000,0.0,8.0,0.597,1.575,7.528,12.600,NaN,0.0,waste_biomass,172.369,0.0,0,-1,0.0,0,0,0,0.0,0,0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,1,0,0,0.0,0.0,0,0.0,0,0,0,0,0,0.0,0,0,0.93,0,0,0.0,0,1,0,0,0,0,0,0,BASN_biomass_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.000000,1,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,Waste_biomass,False,False,0.0,0.0,Biomass,.,.,0.06,0.04,NaN,NaN,NaN,2027
1,BASN,Biomass,2,0.932,0.0,0.833,9.465,157220.000,3.534,1.106,1.291,0.000,0.0,18.0,0.597,1.561,16.776,28.098,NaN,0.0,waste_biomass,172.369,0.0,0,-1,0.0,0,0,0,0.0,0,0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,1,0,0,0.0,0.0,0,0.0,0,0,0,0,0,0.0,0,0,0.93,0,0,0.0,0,1,0,0,0,0,0,0,BASN_biomass_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.000000,2,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,Waste_biomass,False,False,0.0,0.0,Biomass,.,.,0.06,0.04,NaN,NaN,NaN,2027
2,BASN,Conventional Hydroelectric,1,56.580,0.0,0.000,8.843,47048.867,0.000,0.000,0.000,0.000,0.0,44.0,0.356,56.580,2489.52,2489.520,"[0.29812578993421274, 0.2982087954110899, 0.29...",0.0,None,0.000,0.0,0,-1,0.0,0,0,0,0.0,0,0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0,1,0,0,0,0,0,0.0,0.0,0,0.0,0,0,0,0,0,0.0,0,0,0.80,0,0,0.0,0,0,0,0,0,0,0,0,BASN_conventional_hydroelectric_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.389270,3,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,Water,False,False,0.0,0.0,Hydro,.,.,0.05,0.05,NaN,NaN,NaN,2027
3,BASN,Conventional Steam Coal,1,294.550,0.0,96.250,10.572,71499.957,1.879,0.312,0.486,4123.244,0.0,4.0,0.305,294.550,1178.2,1178.200,NaN,16.5,mountain_reference_coal,123.906,0.0,0,-1,0.0,0,0,0,0.0,0,0,0.0,0,0,0,0,0,0,0,1,0,0,0,0,0.0,0,0,0,0,0,0,

In [42]:
b = existing_gens[2027]
b[b.region == 'BASN']

,region,technology,cluster,Cap_Size,Existing_Cap_MWh,minimum_load_mw,Heat_Rate_MMBTU_per_MWh,Fixed_OM_Cost_per_MWyr,Var_OM_Cost_per_MWh,heat_rate_mmbtu_mwh_iqr,heat_rate_mmbtu_mwh_std,fixed_o_m_mw_std,Min_Power,num_units,capacity_factor,unmodified_cap_size,Existing_Cap_MW,unmodified_existing_cap_mw,profile,Start_Fuel_MMBTU_per_MW,Fuel,Start_Cost_per_MW,CapRes_2,RETRO,New_Build,CapRes_6,ESR_16,ESR_15,ESR_12,CapRes_9,ESR_14,Retro2_Inv_Cost_per_MWyr,CapRes_10,Retro1_Efficiency,ESR_9,ESR_13,MinCapTag_4,gen_is_variable,Retro2_Source,Retro1_Inv_Cost_per_MWyr,THERM,Retro3_Efficiency,MinCapTag_1,ESR_3,ESR_8,CO2_Capture_Rate,Num_RETRO_Sources,HYDRO,Retro2_Efficiency,ESR_2,ESR_1,CO2_Capture_Cost_per_Metric_Ton,LDS,CapRes_7,CapRes_4,Retro1_Source,CapRes_5,MinCapTag_2,Num_VRE_Bins,ESR_10,FLEX,ESR_6,CapRes_8,STOR,ESR_7,CapRes_3,MinCapTag_3,MinCapTag_5,CapRes_1,ESR_5,MUST_RUN,VRE,ESR_4,Retro3_Inv_Cost_per_MWyr,Flexible_Demand_Energy_Eff,Retro3_Source,ESR_11,Resource,cost_case,tech_detail,basis_year,Fixed_OM_Cost_per_MWhyr,capex_mw,capex_mwh,wacc_real,Var_OM_Cost_per_MWh_In,cap_recovery_years,Inv_Cost_per_MWyr,Inv_Cost_per_MWhyr,Max_Cap_MW,Max_Cap_MWh,regional_cost_multiplier,area,lcoe,interconnect_annuity,interconnect_capex_mw,m_popden,spur_miles,spur_capex,spur_inv_mwyr,plant_inv_cost_mwyr,backbone_cost_tonne,co2_o_m_mw,co2_pipeline_annuity_mw,co2_pipeline_capex_mw,storage_cost_tonne,capture_rate,extra_ccs_cost_tonne,tonne_co2_captured_mwh,co2_cost_mwh,variable_CF,R_ID,Self_Disch,Eff_Up,Eff_Down,Min_Duration,Max_Duration,Ramp_Up_Percentage,Ramp_Dn_Percentage,Up_Time,Down_Time,Min_Share_percent,Max_Share_percent,Existing_Charge_Cap_MW,Reg_Cost,Rsv_Cost,Reg_Max,Rsv_Max,gen_energy_source,gen_is_cogen,gen_is_baseload,gen_scheduled_outage_rate,gen_forced_outage_rate,gen_type,gen_ccs_capture_efficiency,gen_ccs_energy_load,_gen_scheduled_outage_rate,_gen_forced_outage_rate,Interconnection Capex (test),Interconnection Capex,Total Capex,operating_year
0,BASN,Biomass,1,0.941,0.0,1.138,15.348,157220.000,5.730,1.650,1.385,0.000,0.0,8.0,0.597,1.575,7.528,12.600,NaN,0.0,waste_biomass,172.369,0.0,0,-1,0.0,0,0,0,0.0,0,0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,1,0,0,0.0,0.0,0,0.0,0,0,0,0,0,0.0,0,0,0.93,0,0,0.0,0,1,0,0,0,0,0,0,BASN_biomass_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.000000,1,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,Waste_biomass,False,False,0.0,0.0,Biomass,.,.,0.06,0.04,NaN,NaN,NaN,2027
1,BASN,Biomass,2,0.932,0.0,0.833,9.465,157220.000,3.534,1.106,1.291,0.000,0.0,18.0,0.597,1.561,16.776,28.098,NaN,0.0,waste_biomass,172.369,0.0,0,-1,0.0,0,0,0,0.0,0,0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,1,0,0,0.0,0.0,0,0.0,0,0,0,0,0,0.0,0,0,0.93,0,0,0.0,0,1,0,0,0,0,0,0,BASN_biomass_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.000000,2,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,Waste_biomass,False,False,0.0,0.0,Biomass,.,.,0.06,0.04,NaN,NaN,NaN,2027
2,BASN,Conventional Hydroelectric,1,56.580,0.0,0.000,8.843,47048.867,0.000,0.000,0.000,0.000,0.0,44.0,0.356,56.580,2489.52,2489.520,"[0.29812578993421274, 0.2982087954110899, 0.29...",0.0,None,0.000,0.0,0,-1,0.0,0,0,0,0.0,0,0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0,1,0,0,0,0,0,0.0,0.0,0,0.0,0,0,0,0,0,0.0,0,0,0.80,0,0,0.0,0,0,0,0,0,0,0,0,BASN_conventional_hydroelectric_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.389270,3,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,Water,False,False,0.0,0.0,Hydro,.,.,0.05,0.05,NaN,NaN,NaN,2027
3,BASN,Conventional Steam Coal,1,294.550,0.0,96.250,10.572,71499.957,1.879,0.312,0.486,4123.244,0.0,4.0,0.305,294.550,1178.2,1178.200,NaN,16.5,mountain_reference_coal,123.906,0.0,0,-1,0.0,0,0,0,0.0,0,0,0.0,0,0,0,0,0,0,0,1,0,0,0,0,0.0,0,0,0,0,0,0,

In [43]:
ag = all_gens[2027]
ag[ag.region == 'BASN']

,region,technology,cluster,Cap_Size,Existing_Cap_MWh,minimum_load_mw,Heat_Rate_MMBTU_per_MWh,Fixed_OM_Cost_per_MWyr,Var_OM_Cost_per_MWh,heat_rate_mmbtu_mwh_iqr,heat_rate_mmbtu_mwh_std,fixed_o_m_mw_std,Min_Power,num_units,capacity_factor,unmodified_cap_size,Existing_Cap_MW,unmodified_existing_cap_mw,profile,Start_Fuel_MMBTU_per_MW,Fuel,Start_Cost_per_MW,CapRes_2,RETRO,New_Build,CapRes_6,ESR_16,ESR_15,ESR_12,CapRes_9,ESR_14,Retro2_Inv_Cost_per_MWyr,CapRes_10,Retro1_Efficiency,ESR_9,ESR_13,MinCapTag_4,gen_is_variable,Retro2_Source,Retro1_Inv_Cost_per_MWyr,THERM,Retro3_Efficiency,MinCapTag_1,ESR_3,ESR_8,CO2_Capture_Rate,Num_RETRO_Sources,HYDRO,Retro2_Efficiency,ESR_2,ESR_1,CO2_Capture_Cost_per_Metric_Ton,LDS,CapRes_7,CapRes_4,Retro1_Source,CapRes_5,MinCapTag_2,Num_VRE_Bins,ESR_10,FLEX,ESR_6,CapRes_8,STOR,ESR_7,CapRes_3,MinCapTag_3,MinCapTag_5,CapRes_1,ESR_5,MUST_RUN,VRE,ESR_4,Retro3_Inv_Cost_per_MWyr,Flexible_Demand_Energy_Eff,Retro3_Source,ESR_11,Resource,cost_case,tech_detail,basis_year,Fixed_OM_Cost_per_MWhyr,capex_mw,capex_mwh,wacc_real,Var_OM_Cost_per_MWh_In,cap_recovery_years,Inv_Cost_per_MWyr,Inv_Cost_per_MWhyr,Max_Cap_MW,Max_Cap_MWh,regional_cost_multiplier,area,lcoe,interconnect_annuity,interconnect_capex_mw,m_popden,spur_miles,spur_capex,spur_inv_mwyr,plant_inv_cost_mwyr,backbone_cost_tonne,co2_o_m_mw,co2_pipeline_annuity_mw,co2_pipeline_capex_mw,storage_cost_tonne,capture_rate,extra_ccs_cost_tonne,tonne_co2_captured_mwh,co2_cost_mwh,variable_CF,R_ID,Self_Disch,Eff_Up,Eff_Down,Min_Duration,Max_Duration,Ramp_Up_Percentage,Ramp_Dn_Percentage,Up_Time,Down_Time,Min_Share_percent,Max_Share_percent,Existing_Charge_Cap_MW,Reg_Cost,Rsv_Cost,Reg_Max,Rsv_Max,gen_energy_source,gen_is_cogen,gen_is_baseload,gen_scheduled_outage_rate,gen_forced_outage_rate,gen_type,gen_ccs_capture_efficiency,gen_ccs_energy_load,_gen_scheduled_outage_rate,_gen_forced_outage_rate,Interconnection Capex (test),Interconnection Capex,Total Capex,operating_year
0,BASN,Biomass,1,0.941,0.0,1.138,15.348,157220.000,5.730,1.650,1.385,0.000,0.0,8.0,0.597,1.575,7.528,12.600,NaN,0.0,waste_biomass,172.369,0.0,0,-1,0.0,0,0,0,0.0,0,0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,1,0,0,0.0,0.0,0,0.0,0,0,0,0,0,0.0,0,0,0.93,0,0,0.0,0,1,0,0,0,0,0,0,BASN_biomass_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.000000,1,0.0,1.00,1.00,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,Waste_biomass,False,False,0.0,0.0,Biomass,.,.,0.06,0.04,NaN,NaN,NaN,2027
1,BASN,Biomass,2,0.932,0.0,0.833,9.465,157220.000,3.534,1.106,1.291,0.000,0.0,18.0,0.597,1.561,16.776,28.098,NaN,0.0,waste_biomass,172.369,0.0,0,-1,0.0,0,0,0,0.0,0,0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,1,0,0,0.0,0.0,0,0.0,0,0,0,0,0,0.0,0,0,0.93,0,0,0.0,0,1,0,0,0,0,0,0,BASN_biomass_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.000000,2,0.0,1.00,1.00,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,Waste_biomass,False,False,0.0,0.0,Biomass,.,.,0.06,0.04,NaN,NaN,NaN,2027
2,BASN,Conventional Hydroelectric,1,56.580,0.0,0.000,8.843,47048.867,0.000,0.000,0.000,0.000,0.0,44.0,0.356,56.580,2489.52,2489.520,"[0.29812578993421274, 0.2982087954110899, 0.29...",0.0,None,0.000,0.0,0,-1,0.0,0,0,0,0.0,0,0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0,1,0,0,0,0,0,0.0,0.0,0,0.0,0,0,0,0,0,0.0,0,0,0.80,0,0,0.0,0,0,0,0,0,0,0,0,BASN_conventional_hydroelectric_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.389270,3,0.0,1.00,1.00,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,Water,False,False,0.0,0.0,Hydro,.,.,0.05,0.05,NaN,NaN,NaN,2027
3,BASN,Conventional Steam Coal,1,294.550,0.0,96.250,10.572,71499.957,1.879,0.312,0.486,4123.244,0.0,4.0,0.305,294.550,1178.2,1178.200,NaN,16.5,mountain_reference_coal,123.906,0.0,0,-1,0.0,0,0,0,0.0,0,0,0.0,0,0,0,0,0,0,0,1,0,0,0,0,0.0,0,0,0,

In [ ]:
dfl

In [ ]:
df_time = pd.DataFrame(
    {'year':year,
    't_season':daygroup,
    't_day':hour,
    'timestep':timestep,
    'dayweight':dayweight,
    'SegFrac':segfrac}
)


In [ ]:
dfl.to_csv(results_directory + 'elec_demand_all_years.csv', index=False)
df_time.to_csv(results_directory + 'time.csv', index=False)
for y in reduced_resource_profile.keys():
    reduced_resource_profile[y].to_csv(results_directory + 'capacity_factors_'+str(y)+'.csv', index=False)


In [ ]:
model_regions_gdf = gc.model_regions_gdf
if _settings.get("user_transmission_costs"):
    user_tx_costs = load_user_tx_costs(
        _settings["input_folder"]
        / _settings["user_transmission_costs"],
        _settings["model_regions"],
        _settings.get("target_usd_year"),
    )
    transmission = agg_transmission_constraints(
        pg_engine=pg_engine, settings=_settings
    ).pipe(insert_user_tx_costs, user_costs=user_tx_costs)
else:
    transmission = (
        agg_transmission_constraints(
            pg_engine=pg_engine, settings=_settings
        )
        .pipe(
            transmission_line_distance,
            ipm_shapefile=model_regions_gdf,
            settings=_settings,
            units="mile",
        )
        .pipe(network_line_loss, settings=_settings)
        .pipe(network_reinforcement_cost, settings=_settings)
    )
transmission = (
    transmission.pipe(network_max_reinforcement, settings=_settings)
    .pipe(set_int_cols)
    .pipe(round_col_values)
    .pipe(add_cap_res_network, settings=_settings)
)
zones = _settings["model_regions"]
network_zones = [f"z{n+1}" for n in range(len(zones))]
nz_df = pd.Series(data=network_zones, name="Network_zones")
network = pd.concat([pd.DataFrame(nz_df), transmission], axis=1)
network['WACC'] = settings['transmission_investment_cost']['tx']['wacc']
network['investment_years'] = settings['transmission_investment_cost']['tx']['investment_years']

In [ ]:
network.to_csv(results_directory + 'transmission.csv', index=False)

## Transmission

In [ ]:


# if len(settings['region_aggregations'])>1:
#     transmission = agg_transmission_constraints(pg_engine=pg_engine, settings=settings)
#     model_regions_gdf = load_ipm_shapefile(settings)
#     transmission = transmission_line_distance(
#         trans_constraints_df=transmission,
#         ipm_shapefile=model_regions_gdf,
#         settings=settings,
#     )
#     transmission = network_line_loss(transmission=transmission, settings=settings)
#     transmission = network_reinforcement_cost(transmission=transmission, settings=settings)
#     transmission = network_max_reinforcement(transmission=transmission, settings=settings)
#     transmission = add_cap_res_network(transmission, settings)
#     transmission['source'] = transmission.apply(lambda row: row.transmission_path_name.split('_')[0], axis=1)
#     transmission['destination'] = transmission.apply(lambda row: row.transmission_path_name.split('_')[2], axis=1)
    
# transmission.to_csv(results_directory + 'transmission.csv', index=False)


### Let's make some modifications for Temoa

Let's start by stripping the region name off of the tech name.


In [ ]:
# for y in new_gens.keys():
#     new_gens[y]['Resource'] = new_gens[y]['Resource'].apply(lambda x: re.sub(r'^.*?_', '', x))
#     if not existing_gens[y].empty:
#         existing_gens[y]['Resource'] = existing_gens[y]['Resource'].apply(lambda x: re.sub(r'^.*?_', '', x))
#     all_gens[y]['Resource'] = all_gens[y]['Resource'].apply(lambda x: re.sub(r'^.*?_', '', x))

Now let's include technology lifetimes and retirement dates.


In [ ]:
lifetime_dict = scenario_settings[start_year][scenario]['retirement_ages']
DEFAULT_LIFETIME = 100
DEFAULT_EOL = 2100

In [ ]:
for y in new_gens.keys():
    new_gens[y]['lifetime_yrs'] = DEFAULT_LIFETIME
    new_gens[y]['end_of_life'] = DEFAULT_EOL
    existing_gens[y]['lifetime_yrs'] = DEFAULT_LIFETIME
    existing_gens[y]['end_of_life'] = DEFAULT_EOL
    all_gens[y]['lifetime_yrs'] = DEFAULT_LIFETIME
    all_gens[y]['end_of_life'] = DEFAULT_EOL

In [ ]:
for y in new_gens.keys():
    for t, lt in lifetime_dict.items():
        snake_name = snake_case_str(t)
        resources = all_gens[y].Resource.str.contains(snake_name)
        if resources.sum() > 0:
            all_gens[y].loc[resources, 'lifetime_yrs'] = lt
            all_gens[y].loc[resources, 'end_of_life'] = lt + all_gens[y].loc[resources, 'operating_year']
    # We need to set the existing tech lifetimes to 100. Retirements are dealt with via the MaxCapacity constraint
    all_gens[y].loc[all_gens[y]['Existing_Cap_MW'] > 0, 'lifetime_yrs'] = 100
    all_gens[y].loc[all_gens[y]['Existing_Cap_MW'] > 0, 'end_of_life'] = 2100
    
for y in new_gens.keys():
    for t, lt in lifetime_dict.items():
        snake_name = snake_case_str(t)
        resources = new_gens[y].Resource.str.contains(snake_name)
        if resources.sum() > 0:
            new_gens[y].loc[resources, 'lifetime_yrs'] = lt
            new_gens[y].loc[resources, 'end_of_life'] = lt + new_gens[y].loc[resources, 'operating_year']


for y in new_gens.keys():           
    # We need to set the existing tech lifetimes to 100. Retirements are dealt with via the MaxCapacity constraint
    existing_gens[y].loc[existing_gens[y]['Existing_Cap_MW'] > 0, 'lifetime_yrs'] = 100
    existing_gens[y].loc[existing_gens[y]['Existing_Cap_MW'] > 0, 'end_of_life'] = 2100
            


Now look at special retirement dates provided by EIA... These are under the 'additional_retirements' header in the resources.yml file.

Each entry has the following format: [plant_id_eia, generator_id, planned_retirement_date]

In [ ]:
retirement_dict = scenario_settings[start_year][scenario]['additional_retirements']

## Temporary: Create battery techs with fixed energy-to-power ratios

In [ ]:
for y in new_gens.keys():
    if not existing_gens[y].empty:
        existing_gens[y]['Storage_Duration'] = 'N/A'
        indices = existing_gens[y]['STOR'] == 1
        existing_gens[y].loc[indices, 'Storage_Duration'] = existing_gens[y][indices].Existing_Cap_MWh / existing_gens[y][indices].Existing_Cap_MW


In [ ]:
battery_ratios

In [ ]:
for y in new_gens.keys():
    batt_indices = []
    rows_to_add = []
    ## Need to deal with Pumped Hydro
    ## Below loops should really only be done for NEW resource options...

    new_gens[y]['Storage_Duration'] = 'N/A'


    for idx, row in new_gens[y].iterrows():
        if row.STOR == 0:
            continue
        if idx in new_gens[y].index:
            new_gens[y] = new_gens[y].drop(index=idx)
        for ratio in battery_ratios:
            _row = copy.copy(row)
            _row.Resource = _row.Resource + '_' + str(ratio) + 'hour'
            _row.technology = _row.technology + '_' + str(ratio) + 'hour'
            _row.Storage_Duration = ratio
            # _row.capex_mw += _row.capex_mwh * ratio this is accounted fori
            # _row.Fixed_OM_Cost_per_MWyr += _row.Fixed_OM_Cost_per_MWhyr * ratio
            rows_to_add.append(_row)

    for _r in rows_to_add:
        new_gens[y] = new_gens[y].append(_r, ignore_index = True)

        

In [ ]:
regional_cap_res = create_regional_cap_res(_settings)
regional_cap_res

In [ ]:
# By default, GenX names the regions z1, z2, ....
# We need to change that back to region names in the regional_cap_res dataframe.
regional_cap_res['Network_zones'] = regional_cap_res.index
regional_cap_res

In [ ]:
regional_cap_res.to_csv(results_directory + 'regional_cap_res.csv', index=False)
fuels_price.to_csv(results_directory + 'fuels_price.csv', index=False)
fuels_co2.to_frame().to_csv(results_directory + 'fuels_co2.csv', index=True)
dfl.to_csv(results_directory + 'elec_demand_all_years.csv', index=False)
df_time.to_csv(results_directory + 'time.csv', index=False)

for y in new_gens.keys():
    gen_variability[y]['model_year'] = y
    gen_variability[y].to_csv(results_directory + 'gen_variability_'+str(y)+'.csv', index=False)
    load_curves[y]['model_year'] = y
    load_curves[y].to_csv(results_directory + 'load_curves_'+str(y)+'.csv', index=False)
    reduced_load_profile[y].to_csv(results_directory + 'reduced_load_profile_'+str(y)+'.csv', index=False)
    reduced_resource_profile[y]['model_year'] = y
    reduced_resource_profile[y].to_csv(results_directory + 'reduced_resource_'+str(y)+'.csv', index=False)
    all_gens[y]['tech_index'] = all_gens[y].index
    all_gens[y]['model_year'] = y
    all_gens[y].to_csv(results_directory + 'all_gens_'+str(y)+'.csv', index=False)
    new_gens[y]['model_year'] = y
    new_gens[y].to_csv(results_directory + 'new_gens_'+str(y)+'.csv', index=False)
    existing_gens[y]['model_year'] = y
    existing_gens[y].to_csv(results_directory + 'existing_gens_'+str(y)+'.csv', index=False)


In [ ]:
for y in all_periods:
    if y == all_periods[0]:
        ag = all_gens[y]
        eg = existing_gens[y]
        ng = new_gens[y]
        rlp = reduced_load_profile[y]
    else:
        ag = pd.concat([ag, all_gens[y]])
        eg = pd.concat([eg, existing_gens[y]])
        ng = pd.concat([ng, new_gens[y]])
        rlp = pd.concat([rlp, reduced_load_profile[y]])

In [ ]:
settings.keys()

In [ ]:
df_policies = pd.read_csv(str(settings.get("input_folder")) + '/' + settings.get("emission_policies_fn"))

In [ ]:
str(settings.get("input_folder")) + '/' + settings.get("emission_policies_fn")

In [ ]:
df_policies = pd.read_csv(str(settings.get("input_folder")) + '/conus_existing_policies_rps_ces_20240403.csv')
df_policies.to_csv(results_directory + 'policies' + '.csv')

In [ ]:
ag.to_csv(results_directory + 'all_gens' + '.csv', index=False)
eg.to_csv(results_directory + 'existing_gens' + '.csv', index=False)
ng.to_csv(results_directory + 'new_gens' + '.csv', index=False)
rlp.to_csv(results_directory + 'reduced_load_profile' + '.csv', index=False)
df_policies.to_csv(results_directory + 'policies' + '.csv')

In [ ]:
gen_variability[2050].to_csv('gv2050.csv')
load_curves[2050].to_csv('lc2050.csv')

In [ ]:
dfl

In [ ]:
all_gens.keys()

In [ ]:
df_policies